<a href="https://colab.research.google.com/github/sleeprock/SQL/blob/main/itresume_sql_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Фильтр по наценке и по группе

Дано
Дана таблица analysis:

|  id_an  |  name  |  id_type  |  preanalysis  |  id_cat_an  |  cost_price  |  selling_price  |  term  |  info  |

Задание
С помощью оператора UNION найти анализы из таблицы analysis с наценкой выше 35% или из 7 категории. Повторяющиеся строки необходимо оставить.

Ответ должен содержать 3 поля – name, id_cat_an и рассчитанную Вами наценку в столбце markup.

Примечание: в запросе необходимо использовать именно оператор UNION, а не просто комбинированные условия.

In [ ]:
select a.name, a.id_cat_an, 
(a.selling_price - a.cost_price)/a.cost_price*100 as markup
from analysis a
where (a.selling_price - a.cost_price)/a.cost_price*100 > 35
union all
select a.name, a.id_cat_an, 
(a.selling_price - a.cost_price)/a.cost_price*100 as markup
from analysis a
where a.id_cat_an=7

# Минимальное значение по лексикографическому принципу

Дана таблица analysis:

|  id_an  |  name  |  id_type  |  preanalysis  |  id_cat_an  |  cost_price  |  selling_price  |  term  |  info  |


Дана таблица types_an:

|  id_type  |  name  |  description  |

**Задание**
Выбрать самое минимальное (с лексикографической точки зрения) название типа среди тех анализов, что присутствуют в таблице analysis.

Ответ должен содержать поле name из таблицы types_an.

Примечание: Лексикографический порядок сортировки учитывает регистр букв и их порядковый номер по алфавиту.

In [ ]:
sql = '''select min(t.name) from types_an t
join analysis a on t.id_type = a.id_type
'''

#Транзитивная Связь

Дано
Дана таблица clients:


|  id_cl  |  fio  |  birthday  |  phone  |  email  |  notice  |  discount  |

Дана таблица orders:

|  id_orders  |  order_num  |  order_date  |  id_man  |  id_cl  |  status  |


Дана таблица orderitems:

|  id_orders  |  item_num  |  id_an  |  status  |

Дана таблица analysis:

|  id_an  |  name  |  id_type  |  preanalysis  |  id_cat_an  |  cost_price  |  selling_price  |  term  |  info  |

Дана таблица category_an:

|  id_cat_an  |  name  |  description  |

Задание
Вывести ФИО клиентов, которые когда-либо заказывали анализы из категории «Микроэлементы».

Ответ должен содержать поле fio из таблицы clients.

In [ ]:
#Мой вариант with

sql = '''
with analysys_cat as (
select a.id_an, a.name, ca.name as cat_name from analysis a
join category_an ca on a.id_cat_an = ca.id_cat_an
where ca.name = 'Микроэлементы'
),
sel_orders as (
select * from orderitems oi
join analysys_cat ac on oi.id_an = ac.id_an
 ),
 m_orders as (
 select * from orders o
 join sel_orders so on o.id_orders = so.id_orders
 )
 select t.fio from clients t
join m_orders mo on t.id_cl = mo.id_cl
'''

In [ ]:
#Решение с JOIN 
sql = '''
select cl.fio
from clients cl
join orders o on cl.id_cl = o.id_cl
join orderitems oi on o.id_orders = oi.id_orders
join analysis a on oi.id_an = a.id_an
join category_an ca on a.id_cat_an = ca.id_cat_an
where ca.name = 'Микроэлементы'
'''

# Количество чеков по сотрудникам

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Задание
Найти сотрудников с количеством чеков меньше 8 на основании таблицы transactions.

Ответ должен содержать 2 поля – employee и amount - рассчитанное количество чеков.

In [ ]:
#может быть несколько doc_id для одной транзакции
sql = '''
select t.employee, count(distinct(t.doc_id)) as amount from transactions t
group by t.employee having count(distinct(t.doc_id)) < 8
'''

# Сотрудники с максимальными накоплениями

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Дана таблица types:

+------+--------+
|  id  |  name  |
+------+--------+

In [ ]:
# в таблице types выясняем: 0 - начислено, 1 - списано

#решения

#limit
sql = '''
select t.employee, sum(t.sum) as summ from transactions t
group by t.employee, t.type having t.type = 0 order by summ desc limit 1
'''

#join subquery
sql = '''select a.employee, a.summ from
(select t.employee, sum(t.sum) as summ from transactions t
group by t.employee, t.type having t.type = 0 ) a
join
(
select max(t.summ) from (
select t.employee, sum(t.sum) as summ from transactions t
group by t.employee, t.type having t.type = 0
) t 
) t 
on a.summ = t.max
'''

#window functions
sql = '''with esum as (
select t.employee, sum(t.sum) as summ from transactions t
group by t.employee, t.type having t.type = 0
),
emp_ranked as (
select t.employee, t.summ,
row_number() over(order by t.summ desc) as rn
from esum t
  )
 select t.employee, t.summ from emp_ranked t
 where t.rn = 1
'''


# Начисления с разбивкой по сотрудникам и по типу скидки

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Дана таблица discounts:

+------+---------+
|  id  |  value  |
+------+---------+

Задание
Вывести начисления с разбивкой по сотрудникам и по типу скидки на основании таблицы transactions, а также добавить соответствующее значение скидки из таблицы discounts.

Ответ должен содержать 3 поля – employee, value и рассчитанные начисления в столбце accruals. Результат должен быть упорядочен по сотрудникам в порядке возрастания и по начислениям в порядке убывания.


In [ ]:
#transactions.type  0 начисл, 1 спис

select t.employee,d.value, sum(t.sum) as accruals
from transactions t
join discounts d
on t.disc_id = d.id
where t. type = 0
group by t.employee, d.value
order by t.employee, accruals desc

# Чеки за период дат

Дана таблица transactions:

Задание
Найти количество чеков в период дат 2019-10-25 - 2019-11-02 с группировкой по датам.

Ответ должен содержать 2 поля – date и amount - рассчитанное количество чеков.

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

In [ ]:
sql = '''
select date(t.date), count(distinct t.doc_id) as amount from transactions t
where date(t.date) between '2019-10-25' and '2019-11-02'
group by date(t.date)
'''

# Поиск дублей группировкой

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Задание
В таблице transactions в какой-то момент появилась дублирующаяся строка. Выведите эту строку с помощью оператора GROUP BY.

Ответ должен содержать все поля, кроме поля id.

Примечание: Будем называть строки одинаковыми, если значения всех полей, кроме id, совпадает. ID - автоинкрементный ключ, который по умолчанию при добавлении новой строки сам всегда увеличивается на 1.

In [ ]:
sql = '''
select
id_transaction,
card_id,
maincard_id,
date,
sum,
type,
employee,
doc_id,
cash_id,
shop_id,
doc_type,
disc_id,
disc_source
from transactions t

group by 
id_transaction,
card_id,
maincard_id,
date,
sum,
type,
employee,
doc_id,
cash_id,
shop_id,
doc_type,
disc_id,
disc_source

having count(1) > 1
order by count(1) desc
'''

# Фильтрация агрегированных значений

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Дана таблица discounts:

|  id  |  value  |

Задание
Посчитайте макисмальный % накопления, который был у каждого клиента из таблицы transactions. Посчитайте общую сумму накоплений по каждому клиенту. Посчитайте количество купленных позиций каждым клиентом. Выведите в результате только тех клиентов, у которых максимальный % накопления равен 7 или тех, у кого сумма накоплений больше 20 и куплено менее 5 позиций.

Ответ должен содержать поля:

card_id - идентификатор карты (то есть клиента)
max_perc - максимальный процент накопления
summ - общая сумма накоплений
amount - количество купленных позиций

In [ ]:
sql = '''
select t.card_id, max(d.value) as max_perc, sum(sum) as summ,
count(*) as amount
from transactions t
join discounts d
on t.disc_id = d.id
where type = 0
group by t.card_id
having (max(d.value) = 7) or (sum(sum) > 20 and (count(*)) < 5)
'''

# Второй и третий сотрудник

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Описание полей:

Задание
Вывести сотрудников, которые по величине накоплений находятся на втором и третьем месте на основании таблицы transactions.

Ответ должен содержать столбцы employee и summ - столбец с накоплениями.

In [ ]:
#offset
sql = '''select t.employee, sum(sum) as summ
from transactions t
where t.type = 0
group by t.employee order by summ desc limit 2 offset 1
'''

#window functions
sql = '''select t.employee, t.summ
from
(
select t.employee, sum(sum) as summ,
rank() over (order by sum(sum) desc) as rn
from transactions t
where type = 0
group by t.employee order by sum(sum) desc
) t
where t.rn in (2,3)
'''

# Нарастающий итог

Дано
Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Задание
Вывести id транзакции, сумму текущего списания/начисления для данной транзакции, а также общий баланс списаний/начислений нарастающим итогом.

Ответ должен содержать 3 поля – id_transaction, summ (уже с учетом знака минус, если это списание) и cumsum - рассчитанный нарастающий итог.

In [ ]:
#window function, cte

sql = '''with sum_mod as (
select t.id_transaction,
case when t.type = 0 then t.sum else -(t.sum) end as summ
from transactions t
)
select t.*,
sum(t.summ) over (order by id_transaction) as cumsum
from sum_mod t
'''

#window function
sql = '''select t.id_transaction,
case when t.type = 0 then t.sum else -(t.sum) end as summ,
sum(case when t.type = 0 then t.sum else -(t.sum) end) over (order by id_transaction) as cumsum
from transactions t
'''

# Нумерация строк чека

Дано
Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Задание
Вывести номер чека, дату покупки и порядковый номер строки для конкретного чека.

Ответ должен содержать 3 поля – doc_id, date, num - рассчитанную нумерацию строк.

In [ ]:
#window functions partition by

sql = '''select t.doc_id, date(t.date),
row_number() over (partition by t.doc_id order by t.date) as num
from transactions t
'''


# Ранжирование с разрывами

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Задание
Проранжируйте абсолютные значения начислений и списаний из таблицы transactions в рамках каждого сотрудника. Расчет ранга должен идти по убыванию абсолютного значения. Если для одного сотрудника встречается несколько одинаковых значений, то им должен быть присвоен одинаковый ранг. При этом следующий ранг должен идти с разрывом, например:

+---------+--------+
|  value  |  rank  |
+---------+--------+
|   10    |    1   |
|    5    |    2   |
|    5    |    2   |
|    1    |    4   |
+---------+--------+
Ответ должен содержать 4 поля: employee, type, sum, rnk - рассчитанный ранг.

In [ ]:
slq = '''
select t.employee, t.type, t.sum,
rank() over(partition by t.employee order by t.sum desc) as rnk
from transactions t
'''

# Ранжирование без разрывов

In [ ]:
#условия из предыдущего задания
sql = '''
select t.employee, t.type, t.sum,
dense_rank() over(partition by t.employee order by t.sum desc) as rnk
from transactions t
'''

# [Тестовое Альфа-банк] Покупки телефонов в Туле по месяцам

Дано
Дана таблица клиентов customer:


|  ID_CUSTOMER  |  ADDRESS_1  |  ADDRESS_2  |  COMPANY  |  FIRST_NAME  |  COUNTY_CODE  |  TOWN  |  LAST_NAME  |  POSTCODE  |  TELEPHONE  |


Дана таблица товаров skus:

|  id  |  price  |  category  |

Дана таблица продаж purchases:

|  id  |  created_at  |  user_id  |  sku_id  |

Задание
Необходимо вывести количество людей из Тулы, которые покупали телефоны с разбивкой по месяцам.

Примечание: Телефоны - товары с категорей 2 из таблицы skus.

Важно: Месяца должны быть представлены не в виде цифр, а в виде английских слов.

Примечание: Если один человек купил несколько раз за месяц, считаем это за одну покупку.

Примечание: Предполагается, что в таблице представлены данные только за 1 год.

Примечание: Используемая СУБД - PostgreSQL.

Примечание: Значения в таблице представлены на латинице.

Дополнительная информация
Итоговая таблица должна иметь вид:


|  month  |  people  |

|  june   |     n    |

Важно: Названия столбцов должны в точности совпадать.

Важно: Результат должен быть отсртирован по убыванию значений в столбце people.

In [ ]:
sql = '''select to_char(p.created_at, 'month') as month,
count(distinct p.user_id) as people
from purchases p
join customer c on p.user_id = c.ID_CUSTOMER
join skus s on p.sku_id = s.id
where c.town = 'Tula' and s.category = 2
group by month order by people desc
'''

# Следующее/предыдущее значение

Дана таблица transactions:

|  id  |  id_transaction  |  card_id  |  maincard_id  |  date  |  sum  |  type  |  employee  |  doc_id  |  cash_id  |  shop_id  |  doc_type  |  disc_id  |  disc_source  |

Задание
Для каждой строки вывести предыдущее и следующее значение sum в рамках текущего документа. Если предыдущего/следующего значения нет, вывести NULL.

Примечание: «Предыдущий» и «следующий» обозначают: предыдущая и следующая строка в порядке возрастания столбца id в рамках одного документа.

Ответ должен содержать 5 полей - id, doc_id, sum, ld - следующее значение, lg - предыдущее значение.

In [ ]:
sql = '''
select t.id, t.doc_id, t.sum,
lead(t.sum,1) over (partition by t.doc_id order by t.id) as ld,
lag(t.sum,1) over (partition by t.doc_id order by t.id) as lg
from transactions t
'''

#Прирост начислений

Задание

Для каждого сотрудника посчитайте прирост суммарных начислений по сравнению с предыдущей транзакцией. Списания из расчета необходимо исключить. Если предыдущей транзакции нет, необходимо вывести NULL.

Примечание: «Предыдущая» транзакция определяется по временному признаку.

Ответ должен содержать 6 полей:

employee
id_transaction
dt - дату и время
lg - сумму накоплений в предыдущей транзакции
sm - сумму накоплений в текущей транзакции
inc - значение прироста
Важно: Прирост округлите до 2 знака после запятой.

Ответ должен быть упорядочен по сотрудникам и по дате-времени.

In [ ]:
sql = '''
SELECT t.employee, t.id_transaction, t.dt, t.lg, t.sm,
round((t.sm-t.lg)/(t.lg*1.0), 2) AS inc
from (
 select t.employee, t.id_transaction, t.dt,
 lag(sm) over (partition by t.employee order by t.dt) as lg,
 t.sm
 from (
   select t.employee, t.id_transaction, SUM(sum) AS sm, MIN(t.date) as dt
   from transactions t
   where type = 0
   group by t.employee, t.id_transaction
       ) t
    )t
order by t.employee, t.dt
'''

# Сделать из «широкой» таблицы «длинную»

In [ ]:
#выбираем из широкой несколько подтаблиц, объединяем
sql = '''
select t.name, 'FIO' as key, t.FIO as value
from WideTable t
union
select t.name, 'Phone' as key, t.Phone as value
from WideTable t
union
select t.name, 'Email' as key, t.Email as value
from WideTable t
order by name, key, value
'''

#Найти все пары различных клиентов

In [ ]:
#cross join
sql = '''
with moscow as (
select t.first_name from CUSTOMER t
where t.town = 'Moscow'
)
select m.first_name as customer1, c.first_name as customer2 from moscow m
cross join CUSTOMER c
where c.town = 'Moscow' AND m.first_name != c.first_name
'''
#более аккуратное решение
sql = '''
select c1.first_name as customer1, c2.first_name as customer2
from customer c1
cross join customer c2
where c1.town = 'Moscow' AND c2.town = 'Moscow'
     AND c1.first_name != c2.first_name
'''

#Нарастающий итог продаж
Дано
Есть таблица анализов Analysis:

| an_id | an_name | an_cost | an_price | an_group |

Есть таблица групп анализов Groups:

| gr_id | gr_name | gr_temp |

gr_id - ID группы
gr_name - Название группы
gr_temp - Температурный режим хранения

Есть таблица заказов Orders:

| ord_id | ord_datetime | ord_an  |

ord_id - ID заказа
ord_datetime - Дата и время заказа
ord_an - ID анализа

Задание
Нарастающим итогом вывести увеличение общей суммы продаж с каждым новым заказом в порядке увеличения ID.

Итоговая таблица должна содержать столбцы:

dt - дату заказа
ord_id - ID заказа
cumsum - накопительный итог
Результат отсортируйте по id заказа.

In [ ]:
sql = '''
select date(o.ord_datetime) as dt, o.ord_id ,
sum(an_price) over (order by o.ord_id) as cumsum
from orders o
join Analysis a on a.an_id = o.ord_an
'''

#Третий анализ по количеству продаж

```
Дано
Есть таблица анализов Analysis:

+-------+---------+---------+----------+----------+
| an_id | an_name | an_cost | an_price | an_group |
+-------+---------+---------+----------+----------+
an_id - ID анализа
an_name - Название анализа
an_cost - Себестоимость анализа
an_price - Розничная цена анализа
an_group - Группа анализов
Есть таблица групп анализов Groups:

+-------+---------+---------+
| gr_id | gr_name | gr_temp |
+-------+---------+---------+
gr_id - ID группы
gr_name - Название группы
gr_temp - Температурный режим хранения
Есть таблица заказов Orders:

+--------+--------------+---------+
| ord_id | ord_datetime | ord_an  |
+--------+--------------+---------+
ord_id - ID заказа
ord_datetime - Дата и время заказа
ord_an - ID анализа
Задание
Вывести третий анализ по количеству продаж за весь период.

Результат должен содержать столбцы:

ID анализа
Название анализа
Количество продаж cnt
Ранг анализа в зависимости от продаж rn

```



In [ ]:
sql = '''
select a.an_id, a.an_name, t.cnt,
rank() over(order by t.cnt desc) as rn
from (
 select o.ord_an, count(1) as cnt
 from Orders o
 group by o.ord_an
) t
join Analysis a
on a.an_id = t.ord_an
limit 1 offset 2
'''

#Расположение студентов после пересадки



```
Дана таблица Students, где указана информация о том, где сидит каждый студент:

+----------+----------+
|   seat   |   name   |
+----------+----------+
seat - Номер места студента (автоинкрементное поле, пропусков нет)
name - имя студента
Было принято решение поменять студентов, которые сидят рядом, местами. Напишите SELECT-запрос, который из исходной таблицы сформирует расположение студентов после пересадки.

Примечание: 1 меняется местом со 2, 3 с 4 и так далее. Если студентов нечетное количество, то последний студент не пересаживается, а остается на своем месте.
```



In [ ]:
#мое решение с lag
sql = '''
select
t.seat,
  case when MOD(t.seat, 2) = 0
  then lag(t.name) over (order by t.seat)
  when MOD(t.seat, 2) = 1 AND t.seat != (select MAX(t.seat) from Students t)
  then lead(t.name) over (order by t.seat)
  else t.name
  end as name
from Students t
'''

#решение с изменением номера seat
sql = '''
SELECT
(
  CASE
  WHEN MOD(seat,2)=1 AND seat != (SELECT COUNT(*) FROM Students) THEN seat+1
  WHEN MOD(seat,2)=0 THEN seat-1
  ELSE seat END
),
  name
FROM Students
ORDER BY 1
'''